In [14]:
%reload_ext autoreload
%autoreload 2
if '__file__' not in globals():
    __file__, __name__ = globals()['__vsc_ipynb_file__'], '__ipynb__'
    import types, sys; sys.modules['__ipynb__'] = types.ModuleType('__ipynb__')

import sys, os
if os.path.abspath('.') not in sys.path: sys.path.append(os.path.abspath('.'))

from pathlib import Path
from tqdm import tqdm
from glob import glob

class text_color:
    def __init__(self, *attrs): self.clrs = list(set(attrs))
    def __ror__(self, obj): return f'\33[{";".join(map(str, self.clrs))}m{str(obj)}\33[0m'
    @property
    def bg(self): return text_color( next((c+70 for c in self.clrs if 30 <= c < 38), self.clrs[0]))

bold, italic, underline, strike = [1,3,4,9]
black,red,green,yellow,blue,magenta,cyan,white = [text_color(clr) for clr in range(30,38)]

import inspect
class cout:
    def __ror__(self, obj): print(f'[{inspect.stack()[1].lineno}]'|green, obj)
    def __call__(self, *args, **kwds): print(f'[{inspect.stack()[1].lineno}]'|green, *args, **kwds)
out = cout()

ModuleNotFoundError: spec not found for the module 'IPython.extensions.autoreload'